In [ ]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

In [ ]:
model = init_chat_model("openai:gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### ffmpeg is necessary for audio conversion

Why do I need ffmpeg? It is a universal media converter.

In [ ]:
import subprocess

def is_ffmpeg_installed():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return True
    except FileNotFoundError:
        return False


if is_ffmpeg_installed():
    print("FFmpeg is installed and available in PATH.")
else:
    print("FFmpeg is NOT installed or not in PATH.")

* If you have ffmpeg installed, you are good to go.
* However, if you have it not installed, you should insall it first. It should also be available in your PATH

### On macOS
Just run `brew install ffmpeg`.

### On Windows
Option A — Use ffmpeg.org build
- Download from: https://ffmpeg.org/download.html
- Extract the ZIP
- Add the bin/ folder to PATH:
- Search Environment Variables
- Edit PATH
- Add: `C:\path\to\ffmpeg\bin`

Option B — Use Chocolatey

Just run `choco install ffmpeg`

### Audio converter

In [ ]:
# Doing a test on only one example audio file for transcription

from docling.document_converter import DocumentConverter, AudioFormatOption
from docling.datamodel.pipeline_options import AsrPipelineOptions
from docling.datamodel import asr_model_specs
from docling.datamodel.base_models import InputFormat
from docling.pipeline.asr_pipeline import AsrPipeline
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file if present

def transcribe_audio(audio_path: str) -> str:
    """Transcribe audio file using Whisper ASR."""

    print(f"\n🎙️  Transcribing: {Path(audio_path).name}")
    print("   This may take a moment on first run (downloading Whisper model)...")

    # Configure ASR pipeline with Whisper Turbo
    pipeline_options = AsrPipelineOptions()
    pipeline_options.asr_options = asr_model_specs.WHISPER_TURBO

    # Create converter with ASR configuration
    converter = DocumentConverter(
        format_options={
            InputFormat.AUDIO: AudioFormatOption(
                pipeline_cls=AsrPipeline,
                pipeline_options=pipeline_options,
            )
        }
    )

    # Transcribe
    result = converter.convert(Path(audio_path).resolve())

    # Export to markdown with timestamps
    transcript = result.document.export_to_markdown()

    return transcript

def main():
    print("=" * 60)
    print("Audio Transcription with Docling + Whisper")
    print("=" * 60)

    # Audio file to transcribe
    audio_path = "../documents/raw/founder_bio.mp3"

    print(f"\nInput: {audio_path}")

    try:
        # Transcribe
        transcript = transcribe_audio(audio_path)

        # Display results
        print("\n" + "=" * 60)
        print("TRANSCRIPT OUTPUT")
        print("=" * 60)
        print(transcript[:800])  # Show first 800 characters
        if len(transcript) > 800:
            print("\n... (truncated for display)")

        # Save to file
        output_path = f"../documents/processed/examples/output_{Path(audio_path).stem}.md"
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(transcript)

        print(f"\n✓ Full transcript saved to: {output_path}")
        print(f"✓ Total length: {len(transcript)} characters")

        # Parse timestamp information
        lines = transcript.split('\n')
        timestamp_lines = [line for line in lines if '[time:' in line]
        if timestamp_lines:
            print(f"✓ Found {len(timestamp_lines)} timestamped segments")
            print(f"\nExample timestamp format:")
            print(f"  {timestamp_lines[0][:80]}...")

    except FileNotFoundError:
        print("\n✗ Error: FFmpeg not found!")
        print("\nPlease install FFmpeg:")
        print("  Windows (Chocolatey): choco install ffmpeg")
        print("  Windows (Conda):      conda install -c conda-forge ffmpeg")
        print("  macOS:                brew install ffmpeg")
        print("  Linux:                apt-get install ffmpeg")

    except Exception as e:
        print(f"\n✗ Error: {e}")
        print("\nMake sure:")
        print("  1. FFmpeg is installed and in PATH")
        print("  2. Audio file exists and is readable")
        print("  3. Audio format is supported (MP3, WAV, M4A, FLAC)")

In [ ]:
main()

### Text converter

In [ ]:
# Doing a test on three sample text docs

def process_text_docs(file_path: str) -> dict:
    """Process a single document and return metadata."""
    try:
        print(f"\n📄 Processing: {Path(file_path).name}")

        # instantiate the converter
        converter = DocumentConverter()

        # Convert document
        result = converter.convert(file_path)

        # Export to markdown
        markdown = result.document.export_to_markdown()

        # Get document info
        doc_info = {
            'file': Path(file_path).name,
            'format': Path(file_path).suffix,
            'status': 'Success',
            'markdown_length': len(markdown),
            'preview': markdown[:200].replace('\n', ' ')
        }

        # Save output
        output_file = f"../documents/processed/examples/output_{Path(file_path).stem}.md"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(markdown)

        doc_info['output_file'] = output_file

        print(f"   ✓ Converted successfully")
        print(f"   ✓ Output: {output_file}")

        return doc_info

    except Exception as e:
        print(f"   ✗ Error: {e}")
        return {
            'file': Path(file_path).name,
            'format': Path(file_path).suffix,
            'status': 'Failed',
            'error': str(e)
        }

def main():
    print("=" * 60)
    print("Multi-Format Document Processing with Docling")
    print("=" * 60)

    # List of documents to process
    documents = [
        "../documents/raw/technical-architecture-guide.pdf",
        "../documents/raw/meeting-notes-2025-01-08.docx",
        "../documents/raw/company-overview.md",
    ]

    # Process all documents
    results = []
    for doc_path in documents:
        result = process_text_docs(doc_path)
        results.append(result)

    # Summary
    print("\n" + "=" * 60)
    print("CONVERSION SUMMARY")
    print("=" * 60)

    for result in results:
        status_icon = "✓" if result['status'] == 'Success' else "✗"
        print(f"{status_icon} {result['file']} ({result['format']})")
        if result['status'] == 'Success':
            print(f"   Length: {result['markdown_length']} chars")
            print(f"   Preview: {result['preview']}...")
        else:
            print(f"   Error: {result.get('error', 'Unknown')}")
        print()

    success_count = sum(1 for r in results if r['status'] == 'Success')
    print(f"Converted {success_count}/{len(results)} documents successfully")

In [ ]:
main()